In [1]:
import os
import pandas as pd
from pathlib import Path

In [2]:
home = '/home/sshankar'
ibc_folder = 'ibc'

In [3]:
# The sessions.csv file contains information on which task 
# was acquired in which session
# Use it to also get the list of subjects and sessions

sessfile = os.path.join(home, ibc_folder, 'public_analysis_code/ibc_data/sessions.csv')
sess_df = pd.read_csv(sessfile)
subjects = sess_df.subject
sessions = sess_df.columns[1:]

In [4]:
# What is the task of interest?
task = 'audio'

In [5]:
subjects

0     sub-01
1     sub-02
2     sub-04
3     sub-05
4     sub-06
5     sub-07
6     sub-08
7     sub-09
8     sub-11
9     sub-12
10    sub-13
11    sub-14
12    sub-15
Name: subject, dtype: object

In [8]:
subs = ['sub-01', 'sub-04', 'sub-05', 'sub-06', 'sub-07', 'sub-08', 
        'sub-09', 'sub-11', 'sub-12', 'sub-13', 'sub-14', 'sub-15']
# raiders1 = [16, 13, 13, 13, 14, 14, 14, 14, 13, 14]
# raiders2 = [17, 14, 14, 14, 15, 15, 15, 15, 14, 15]

In [6]:
# For each subject, find which sessions contain the task of interest
task_sess = []

for i in range(len(subjects)):
#     if subjects[i] in subs:
    ser = sess_df.iloc[i,:]
    ids = ser.str.contains(task)==True
    sess = ser.loc[ids].keys().tolist()
    task_sess.append(sess)

In [7]:
task_sess

[[],
 [],
 ['ses-22', 'ses-23'],
 ['ses-21', 'ses-23'],
 ['ses-22', 'ses-23'],
 ['ses-21', 'ses-22'],
 [],
 ['ses-23', 'ses-24'],
 ['ses-23', 'ses-24'],
 ['ses-23', 'ses-24'],
 ['ses-22', 'ses-23'],
 ['ses-22', 'ses-23'],
 ['ses-22', 'ses-23']]

In [8]:
data_dir_local = os.path.join(home, task, '3mm')
data_dir_drago = 'sshankar@drago:/storage/store/data/ibc/3mm/'
events_dir_drago = 'sshankar@drago:/storage/store/data/ibc/sourcedata/'

In [15]:
# Create directories for subjects and sessions
for i, sub in enumerate(subjects):
    for ses in task_sess[i]:
        ses_path = os.path.join(data_dir_local, sub, ses)
        if not os.path.isdir(ses_path):
            os.makedirs(ses_path)

In [16]:
# To copy data files
for i, sub in enumerate(subjects):
    for ses in task_sess[i]:
        ses_path = os.path.join(data_dir_local, sub, ses)
        os.chdir(ses_path)
        remote_data_file = os.path.join(data_dir_drago, sub, ses, 'func/wrd*')
        os.system('scp %s .' % (remote_data_file) )

In [10]:
# To copy events files
for i, sub in enumerate(subjects):
    for ses in task_sess[i]:
        ses_path = os.path.join(data_dir_local, sub, ses)
        os.chdir(ses_path)
        remote_data_file = os.path.join(events_dir_drago, sub, ses, 'func/*tsv')
        os.system('scp %s .' % (remote_data_file) )